In [1]:
import sys
sys.path.append('../../src/meta_rule/')

from lnn_operators import and_lukasiewicz, or_lukasiewicz, negation
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

torch.manual_seed(100)

## Train and Val

In [4]:
# train and val

df_train_val = pd.read_csv("train.csv")
df_train_val = df_train_val.loc[22:25]

features_train_val = np.array([np.fromstring(s[1:-1], dtype=np.float, sep=', ') for s in df_train_val.Features.values])

#to train a xor we need its truth table
X_train_val = torch.from_numpy(features_train_val).float()
print(X_train_val, X_train_val.shape)
#the target values for each row in the truth table (xor)
Y_train_val = torch.from_numpy(df_train_val.Label.values).float()
print(Y_train_val, Y_train_val.shape)
# mention_labels (cannot convert string explicitly)
mention_labels_train_val = df_train_val.Mention_label.values
print(mention_labels_train_val)

x_train, x_val, y_train, y_val, m_labels_train, m_labels_val = \
    train_test_split(X_train_val, Y_train_val, mention_labels_train_val, test_size=0.2,train_size=0.8, random_state=100)


tensor([[0.8444, 0.5000, 0.3333, 0.6638, 0.5465, 1.0000],
        [0.4815, 0.0000, 0.0000, 0.4734, 0.5313, 0.4000],
        [0.6375, 0.0000, 0.1250, 0.4944, 0.5258, 1.0000],
        [0.4815, 0.0000, 0.0000, 0.4476, 0.5201, 0.4000]]) torch.Size([4, 6])
tensor([1., 0., 0., 0.]) torch.Size([4])
['GMT;GMT Games' 'GMT;UTC+10:00' 'GMT;Giant Magellan Telescope'
 'GMT;UTC+12:00']


## Test

In [5]:
# test
df_test = pd.read_csv("test.csv")

features_test = np.array([np.fromstring(s[1:-1], dtype=np.float, sep=', ') for s in df_test.Features.values])

x_test = torch.from_numpy(features_test).float()
print(x_test, x_test.shape)
y_test = torch.from_numpy(df_test.Label.values).float()
print(y_test, y_test.shape)
m_labels_test = df_test.Mention_label.values
print(m_labels_test)

tensor([[0.8903, 0.6667, 0.5625, 0.8557, 1.0000, 0.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 0.7529, 0.0000],
        [0.9385, 0.6667, 0.6923, 0.9036, 0.1000, 1.0000],
        ...,
        [0.0000, 0.5000, 0.5556, 0.7310, 0.1000, 0.0000],
        [0.4256, 0.3333, 0.3846, 0.5935, 0.1000, 0.0000],
        [0.6444, 0.3333, 0.3333, 0.5635, 0.1000, 1.0000]]) torch.Size([23846, 6])
tensor([0., 1., 0.,  ..., 0., 0., 0.]) torch.Size([23846])
['Düsseldorf Airport;Düsseldorf International Airport'
 'Düsseldorf Airport;Düsseldorf Airport'
 'Düsseldorf Airport;Düsseldorf Airport station' ... 'Pluto;HMS Pluto'
 'Pluto;Terry Pluto 1' 'Pluto;The Pluto Files']


In [6]:
print('train', x_train.shape)
print('val', x_test.shape)

train torch.Size([3, 6])
val torch.Size([23846, 6])


In [7]:
class PureNameLNN(nn.Module):
    def __init__(self, alpha, arity, slack=None):
        super(PureNameLNN, self).__init__()
        self.threshold = 0.5
        
        self.sim_disjunction_or_ops = nn.ModuleList([or_lukasiewicz(alpha, arity, slack) for i in range(3)])
        self.predicate_and = and_lukasiewicz(alpha, arity, slack)
    
    def forward(self, x, mention_labels=None):
        """
            x: scores['jw'], scores['jacc'], scores['lev'], scores['spacy'], 
               normalized_ref_scores[ref_idx], normalized_ctx_scores[ctx_idx]
        """
        yhat = None
        
        ####### RULE 1: lookup predicate #######
        lookup_features = x[:,4].view(-1, 1)
#         print("lookup_features", lookup_features)
        
        
        ####### RULE 2: similarity predicate(mention==label AND Jacc(m, lb) AND Lev(m, lb) AND Jaro(m, lb)) #######
        feature_list = []
        # rule 2 (1) mention==label
        mentions = np.array([m[0].lower() for m in mention_labels])
        labels = np.array([m[1].lower() for m in mention_labels])
        exact_match_features = torch.from_numpy(np.array(mentions == labels).astype(float)).float().view(-1,1)
        feature_list.append(exact_match_features)
#         print("exact_match_features", exact_match_features)
        
        # rule 2 (2) Jacc(mention, label)
        jacc_features = x[:, 1].view(-1,1)
#         jacc_features = torch.clamp(jacc_features, min=self.threshold, max=1.0)
        jacc_features_ = torch.where(jacc_features>=self.threshold, jacc_features, torch.zeros_like(jacc_features))
        feature_list.append(jacc_features_)
#         print("jacc_features", jacc_features)
#         print("jacc_features*mask", jacc_features_)
        
        # rule 2 (3) Lev(mention, label)
        lev_features = x[:, 2].view(-1,1)
        lev_features_ = torch.where(lev_features>=self.threshold, lev_features, torch.zeros_like(lev_features))
        feature_list.append(lev_features_)
#         print("lev_features", lev_features)
#         print("lev_features*mask", lev_features_)
        
        # rule 2 (4) Jaro(mention, label)
        jaro_features = x[:, 0].view(-1,1)
        jaro_features_ = torch.where(jaro_features>=self.threshold, jaro_features, torch.zeros_like(jaro_features))
        feature_list.append(jaro_features_)
#         print("jaro_features", jaro_features)
#         print("jaro_features*mask", jaro_features_)
        
        # disjunction of (1) to (4)
        disjunction_result = feature_list[0]
        for i in range(0, 3):
            disjunction_result = self.sim_disjunction_or_ops[i](torch.cat((disjunction_result, feature_list[i+1]), 1))
#             print("disjunction_result", disjunction_result)
        
        # RULE 1 + RULE 2
        yhat = self.predicate_and(torch.cat((lookup_features, disjunction_result), 1))
#         print('yhat', yhat)
        return yhat

In [6]:
class ContextLNN(nn.Module):
    def __init__(self, alpha, arity, slack=None):
        super(ContextLNN, self).__init__()
        self.threshold = 0.5
        
        self.sim_disjunction_or_ops = nn.ModuleList([or_lukasiewicz(alpha, arity, slack) for i in range(4)])
        self.predicate_and_ops = nn.ModuleList([and_lukasiewicz(alpha, arity, slack) for i in range(2)])
    
    def forward(self, x, mention_labels=None):
        """
            x: scores['jw'], scores['jacc'], scores['lev'], scores['spacy'], 
               normalized_ref_scores[ref_idx], normalized_ctx_scores[ctx_idx]
        """
        yhat = None
        ####### RULE 1: lookup predicate #######
        lookup_features = x[:,4].view(-1, 1)
#         print("lookup_features", lookup_features)
        
        ####### RULE 3: contains predicate #######
        context_features = x[:,5].view(-1, 1)
#         print("context_features", context_features)
        # context mask
        context_mask = context_features >= 0.25
#         print("context_mask", context_mask)
        
        ####### RULE 2: similarity predicate(mention==label AND Jacc(m, lb) AND Lev(m, lb) AND Jaro(m, lb)) #######
        # check: https://github.ibm.com/IBM-Research-AI/enhanced_amr/blob/5501e3af41794353ed9bb147320666622474171a/entity_linking.py#L295
        feature_list = []
        # rule 2 (1) mention==label
        mentions = np.array([m[0].lower() for m in mention_labels])
        labels = np.array([m[1].lower() for m in mention_labels])
        exact_match_features = torch.from_numpy(np.array(mentions == labels).astype(float)).float().view(-1,1)
        feature_list.append(exact_match_features)
        
        # rule 2 (2) Jacc(mention, label)
        jacc_features = x[:, 1].view(-1,1)
        jacc_features_ = torch.where(jacc_features>=self.threshold-0.3, jacc_features, torch.zeros_like(jacc_features))
        feature_list.append(jacc_features_*context_mask)
        
        # rule 2 (3) Lev(mention, label)
        lev_features = x[:, 2].view(-1,1)
        lev_features_ = torch.where(lev_features>=self.threshold-0.3, lev_features, torch.zeros_like(lev_features))
        feature_list.append(lev_features_*context_mask)
#         print("lev_features", lev_features)
#         print("lev_features*mask", lev_features_)
        
        # rule 2 (4) Jaro(mention, label)
        jaro_features = x[:, 0].view(-1,1)
        jaro_features_ = torch.where(jaro_features>=self.threshold-0.3, jaro_features, torch.zeros_like(jaro_features))
        feature_list.append(jaro_features_*context_mask)
        
        # disjunction of (1) to (4)
        disjunction_result = feature_list[0]
        for i in range(0, 3):
            disjunction_result = self.sim_disjunction_or_ops[i](torch.cat((disjunction_result, feature_list[i+1]), 1))
        
        # RULE 1 + RULE 2
        r1_r2_res = self.predicate_and_ops[0](torch.cat((lookup_features, disjunction_result), 1))
        yhat = self.predicate_and_ops[1](torch.cat((r1_r2_res, context_features), 1))
        return yhat

In [7]:
class ComplexRuleLNN(nn.Module):
    def __init__(self, alpha, arity, slack=None):
        super(ComplexRuleLNN, self).__init__()
        
        self.pureNameRule = PureNameLNN(alpha, arity, None)
        self.contextRule = ContextLNN(alpha, arity, None)
        self.rule_or_ops = nn.ModuleList([or_lukasiewicz(alpha, arity, slack) for i in range(2)])
    
    def forward(self, x, mention_labels=None):
        
        yhat = None
        
        pure_res = self.pureNameRule(x, mention_labels)
        context_res = self.contextRule(x, mention_labels)
#         print('context_res', context_res)
        pure_context_res = self.rule_or_ops[0](torch.cat((pure_res, context_res), 1))
#         print('pure_context_res', pure_context_res)
        return pure_context_res

# Train PureNameLNN

In [8]:
# Sanity Check
model = PureNameLNN(0.8, 2, False)
print(model(x_train, m_labels_train))

loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

def evaluate(eval_model, x_eval, y_eval, m_labels_eval):
    eval_model.eval()
    with torch.no_grad():
        val_pred = eval_model(x_eval, m_labels_eval)
        loss = loss_fn(val_pred, y_eval)
        val_pred_ = val_pred > 0.5
        print("val loss", loss)
        prec, recall, f1, _ = precision_recall_fscore_support(y_eval, val_pred_, average='macro')
        print("f1 w/ 0.5 threshold", f1)
    return loss, f1, val_pred
    

tensor([[9.4524e-03],
        [1.7927e-07],
        [9.9078e-01],
        ...,
        [5.0680e-03],
        [1.0939e-05],
        [8.2041e-04]], grad_fn=<SWhereBackward>)


/Users/hjian42/opt/anaconda3/envs/lnn/lib/python3.8/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [12]:
best_pred = None
best_val_f1, best_val_loss = 0, 1000

for iter in range(200):

    model.train(True)
    optimizer.zero_grad()

    yhat = model(x_train, m_labels_train)
    loss = loss_fn(yhat, y_train)

    print("Iteration " + str(iter) + ": " + str(loss.item()))
    loss.backward()
    optimizer.step()

    val_loss, val_f1, val_pred = evaluate(model, x_val, y_val, m_labels_val)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_val_f1 = val_f1
        best_pred = val_pred
        torch.save(model.state_dict(), "best_PureNameLNN.pt")


Iteration 0: 0.03993843123316765
val loss tensor(0.0404)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 1: 0.03992585465312004
val loss tensor(0.0404)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 2: 0.0399133674800396
val loss tensor(0.0404)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 3: 0.039900947362184525
val loss tensor(0.0403)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 4: 0.039888542145490646
val loss tensor(0.0403)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 5: 0.03987625241279602
val loss tensor(0.0403)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 6: 0.03986399993300438
val loss tensor(0.0403)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 7: 0.03985206410288811
val loss tensor(0.0403)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 8: 0.039843134582042694
val loss tensor(0.0403)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 9: 0.03983452543616295
val loss tensor(0.0403)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 10: 0.03982608765363693
val 

val loss tensor(0.0400)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 88: 0.03926197066903114
val loss tensor(0.0400)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 89: 0.039257876574993134
val loss tensor(0.0400)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 90: 0.0392538383603096
val loss tensor(0.0400)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 91: 0.03924982249736786
val loss tensor(0.0400)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 92: 0.0392458438873291
val loss tensor(0.0399)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 93: 0.039241887629032135
val loss tensor(0.0399)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 94: 0.039237964898347855
val loss tensor(0.0399)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 95: 0.039234090596437454
val loss tensor(0.0399)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 96: 0.039230335503816605
val loss tensor(0.0399)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 97: 0.03922651708126068
val loss tensor(0.0399)
f1 

val loss tensor(0.0398)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 176: 0.03901956230401993
val loss tensor(0.0398)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 177: 0.039017580449581146
val loss tensor(0.0398)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 178: 0.03901569917798042
val loss tensor(0.0398)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 179: 0.03901384025812149
val loss tensor(0.0398)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 180: 0.039011966437101364
val loss tensor(0.0398)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 181: 0.03901011869311333
val loss tensor(0.0398)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 182: 0.03900829330086708
val loss tensor(0.0398)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 183: 0.03900645300745964
val loss tensor(0.0398)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 184: 0.039004623889923096
val loss tensor(0.0398)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 185: 0.039002858102321625
val loss tensor(

In [13]:
# tune on val set

print("Val -- The best f1 is {} w/ naive threshold 0.5".format(best_val_f1))

best_tuned_threshold = 0.5
best_tuned_f1 = best_val_f1

for threshold_ in np.linspace(0.0, 1.0, num=1000):
    y_val_preds = best_pred >= threshold_
    prec, recall, f1, _ = precision_recall_fscore_support(y_val, y_val_preds, average='macro')
    if f1 > best_tuned_f1:
        best_tuned_threshold = threshold_
        best_tuned_f1 = f1
print("Val -- After tuning, the best f1 is {} w/ threshold {}".format(best_tuned_f1, best_tuned_threshold))

Val -- The best f1 is 0.7727933173225794 w/ naive threshold 0.5
Val -- After tuning, the best f1 is 0.9029920974347565 w/ threshold 0.997997997997998


# test data

In [113]:
bestModel = PureNameLNN(0.8, 2, False)
bestModel.load_state_dict(torch.load("best_PureNameLNN.pt"))
bestModel.eval()

with torch.no_grad():
    test_pred = bestModel(x_test, m_labels_test)
    test_pred = test_pred >= best_tuned_threshold
    prec, recall, f1, _ = precision_recall_fscore_support(y_test, test_pred, average='macro')
    print("Test -- f1 is {} w/ threshold {}".format(f1, best_tuned_threshold))

Test -- f1 is 0.8817386090617398 w/ threshold 0.997997997997998


In [148]:
for name, mod in bestModel.named_children():
    print("========={}=========".format(name))
    if type(mod) == nn.ModuleList:
        for each_op in mod:
            print(each_op.AND.cdd())
    else:
        print(mod, mod.cdd())

=========sim_disjunction_or_ops=========
(tensor(8.6251, grad_fn=<SelectBackward>), tensor([19.7417, 10.8246], grad_fn=<SliceBackward>))
(tensor(8.4585, grad_fn=<SelectBackward>), tensor([19.1825, 11.0743], grad_fn=<SliceBackward>))
(tensor(7.0526, grad_fn=<SelectBackward>), tensor([16.0262,  8.7171], grad_fn=<SliceBackward>))
(tensor(4.1313, grad_fn=<SelectBackward>), tensor([7.7373, 6.6682], grad_fn=<SliceBackward>))
=========predicate_and=========
and_lukasiewicz(
  (cdd): cdd_lnn()
) (tensor(2.8722, grad_fn=<SelectBackward>), tensor([6.9262, 3.3761], grad_fn=<SliceBackward>))


In [118]:
# beta, argument_wts = bestModel.predicate_and.cdd()
# print("beta (post-training): " + str(beta.item()))
# print("argument weights (post-training): " + str(argument_wts.detach()))

# for each_op in bestModel.sim_disjunction_or_ops:
#     #lets check the LNN conjunction parameters post-training
#     #do these look different from the pre-training settings?
#     beta, argument_wts = each_op.AND.cdd()
#     print("beta (post-training): " + str(beta.item()))
#     print("argument weights (post-training): " + str(argument_wts.detach()))

beta (post-training): 2.8722004890441895
argument weights (post-training): tensor([6.9262, 3.3761])
beta (post-training): 8.625083923339844
argument weights (post-training): tensor([19.7417, 10.8246])
beta (post-training): 8.45850658416748
argument weights (post-training): tensor([19.1825, 11.0743])
beta (post-training): 7.052649021148682
argument weights (post-training): tensor([16.0262,  8.7171])
beta (post-training): 4.1312785148620605
argument weights (post-training): tensor([7.7373, 6.6682])


# Other models

In [ ]:
## ContextLNN

In [14]:
# Sanity Check
model = ContextLNN(0.8, 2, False)
print(model(x_train, m_labels_train))

loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

def evaluate(eval_model, x_eval, y_eval, m_labels_eval):
    eval_model.eval()
    with torch.no_grad():
        val_pred = eval_model(x_eval, m_labels_eval)
        loss = loss_fn(val_pred, y_eval)
        val_pred_ = val_pred > 0.5
        print("val loss", loss)
        prec, recall, f1, _ = precision_recall_fscore_support(y_eval, val_pred_, average='macro')
        print("f1 w/ 0.5 threshold", f1)
    return loss, f1, val_pred


best_pred = None
best_val_f1, best_val_loss = 0, 10000

for iter in range(200):

    model.train(True)
    optimizer.zero_grad()

    yhat = model(x_train, m_labels_train)
    loss = loss_fn(yhat, y_train)

    print("Iteration " + str(iter) + ": " + str(loss.item()))
    loss.backward()
    optimizer.step()

    val_loss, val_f1, val_pred = evaluate(model, x_val, y_val, m_labels_val)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_val_f1 = val_f1
        best_pred = val_pred
        torch.save(model.state_dict(), "best_ContextLNN.pt")


tensor([[2.9200e-04],
        [1.5054e-07],
        [9.9943e-01],
        ...,
        [2.6379e-04],
        [1.0349e-05],
        [2.5376e-04]], grad_fn=<SWhereBackward>)
Iteration 0: 0.2859273850917816
val loss tensor(0.2682)
f1 w/ 0.5 threshold 0.5848549075800495
Iteration 1: 0.2410481721162796
val loss tensor(0.2264)
f1 w/ 0.5 threshold 0.5960588363785881
Iteration 2: 0.20673899352550507
val loss tensor(0.1875)
f1 w/ 0.5 threshold 0.6139278208619465
Iteration 3: 0.1752336174249649
val loss tensor(0.1554)
f1 w/ 0.5 threshold 0.6279422651402675
Iteration 4: 0.1513768881559372
val loss tensor(0.1329)
f1 w/ 0.5 threshold 0.6470300283758199
Iteration 5: 0.1327628642320633
val loss tensor(0.1150)
f1 w/ 0.5 threshold 0.6585167134759234
Iteration 6: 0.11826548725366592
val loss tensor(0.1046)
f1 w/ 0.5 threshold 0.6729966604011413
Iteration 7: 0.10744988918304443
val loss tensor(0.0941)
f1 w/ 0.5 threshold 0.6797538720201313
Iteration 8: 0.09956284612417221
val loss tensor(0.0883)
f1 w/ 0.

Iteration 87: 0.04431500658392906
val loss tensor(0.0404)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 88: 0.04427635297179222
val loss tensor(0.0404)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 89: 0.04423997551202774
val loss tensor(0.0404)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 90: 0.04420547932386398
val loss tensor(0.0403)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 91: 0.044172924011945724
val loss tensor(0.0403)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 92: 0.04414163529872894
val loss tensor(0.0402)
f1 w/ 0.5 threshold 0.7727933173225794
Iteration 93: 0.044053349643945694
val loss tensor(0.0402)
f1 w/ 0.5 threshold 0.7758931189010344
Iteration 94: 0.043939944356679916
val loss tensor(0.0401)
f1 w/ 0.5 threshold 0.7758931189010344
Iteration 95: 0.04386923462152481
val loss tensor(0.0401)
f1 w/ 0.5 threshold 0.7758931189010344
Iteration 96: 0.04381221905350685
val loss tensor(0.0400)
f1 w/ 0.5 threshold 0.7758931189010344
Iteration 97: 0.04376303777

Iteration 177: 0.03976478427648544
val loss tensor(0.0368)
f1 w/ 0.5 threshold 0.7960179928064651
Iteration 178: 0.039749402552843094
val loss tensor(0.0368)
f1 w/ 0.5 threshold 0.7960179928064651
Iteration 179: 0.03973430022597313
val loss tensor(0.0368)
f1 w/ 0.5 threshold 0.7960179928064651
Iteration 180: 0.03972012177109718
val loss tensor(0.0368)
f1 w/ 0.5 threshold 0.7960179928064651
Iteration 181: 0.03970617800951004
val loss tensor(0.0366)
f1 w/ 0.5 threshold 0.7960179928064651
Iteration 182: 0.039692845195531845
val loss tensor(0.0366)
f1 w/ 0.5 threshold 0.7960179928064651
Iteration 183: 0.03968033939599991
val loss tensor(0.0365)
f1 w/ 0.5 threshold 0.7960179928064651
Iteration 184: 0.03966781497001648
val loss tensor(0.0364)
f1 w/ 0.5 threshold 0.7960179928064651
Iteration 185: 0.03964666277170181
val loss tensor(0.0364)
f1 w/ 0.5 threshold 0.7960179928064651
Iteration 186: 0.0396248921751976
val loss tensor(0.0364)
f1 w/ 0.5 threshold 0.7960179928064651
Iteration 187: 0.03

In [18]:
# tune on val set

print("Val -- The best f1 is {} w/ naive threshold 0.5".format(best_val_f1))

best_tuned_threshold = 0.5
best_tuned_f1 = best_val_f1

for threshold_ in np.linspace(0.0, 1.0, num=1000):
    y_val_preds = best_pred >= threshold_
    prec, recall, f1, _ = precision_recall_fscore_support(y_val, y_val_preds, average='macro')
    if f1 > best_tuned_f1:
        best_tuned_threshold = threshold_
        best_tuned_f1 = f1
print("Val -- After tuning, the best f1 is {} w/ threshold {}".format(best_tuned_f1, best_tuned_threshold))


bestModel = ContextLNN(0.8, 2, False)
bestModel.load_state_dict(torch.load("best_ContextLNN.pt"))
bestModel.eval()

with torch.no_grad():
    test_pred = bestModel(x_test, m_labels_test)
    test_pred = test_pred >= best_tuned_threshold
    prec, recall, f1, _ = precision_recall_fscore_support(y_test, test_pred, average='macro')
    print("Test -- f1 is {} w/ threshold {}".format(f1, best_tuned_threshold))
    print("prec, recall, f1", prec, recall, f1)

Val -- The best f1 is 0.7960179928064651 w/ naive threshold 0.5
Val -- After tuning, the best f1 is 0.8071826023682409 w/ threshold 0.992992992992993
Test -- f1 is 0.811596787435352 w/ threshold 0.992992992992993
prec, recall, f1 0.7739705893126639 0.8616414428353842 0.811596787435352


In [153]:
for name, mod in bestModel.named_children():
    print("========={}=========".format(name))
    if name == 'sim_disjunction_or_ops':
        for each_op in mod:
            print(each_op.AND.cdd())
    elif name == 'predicate_and_ops':
        for each_op in mod:
            print(each_op.cdd())
    else:
        print(mod, mod.cdd())

=========sim_disjunction_or_ops=========
(tensor(5.8912, grad_fn=<SelectBackward>), tensor([12.1745,  8.3585], grad_fn=<SliceBackward>))
(tensor(8.8607, grad_fn=<SelectBackward>), tensor([19.9964, 11.0276], grad_fn=<SliceBackward>))
(tensor(7.7926, grad_fn=<SelectBackward>), tensor([17.1775,  9.7577], grad_fn=<SliceBackward>))
(tensor(4.7304, grad_fn=<SelectBackward>), tensor([8.7144, 7.7403], grad_fn=<SliceBackward>))
=========predicate_and_ops=========
(tensor(5.3318, grad_fn=<SelectBackward>), tensor([15.9246,  6.5601], grad_fn=<SliceBackward>))
(tensor(1.8147, grad_fn=<SelectBackward>), tensor([2.9074, 2.0872], grad_fn=<SliceBackward>))


In [ ]:
# Complex

In [154]:
# Sanity Check
model = ComplexRuleLNN(0.8, 2, False)
print(model(x_train, m_labels_train))

loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

def evaluate(eval_model, x_eval, y_eval, m_labels_eval):
    eval_model.eval()
    with torch.no_grad():
        val_pred = eval_model(x_eval, m_labels_eval)
        loss = loss_fn(val_pred, y_eval)
        val_pred_ = val_pred > 0.5
        print("val loss", loss)
        prec, recall, f1, _ = precision_recall_fscore_support(y_eval, val_pred_, average='macro')
        print("f1 w/ 0.5 threshold", f1)
    return loss, f1, val_pred


best_pred = None
best_val_f1, best_val_loss = 0, 10000

for iter in range(200):

    model.train(True)
    optimizer.zero_grad()

    yhat = model(x_train, m_labels_train)
    loss = loss_fn(yhat, y_train)

    print("Iteration " + str(iter) + ": " + str(loss.item()))
    loss.backward()
    optimizer.step()

    val_loss, val_f1, val_pred = evaluate(model, x_val, y_val, m_labels_val)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_val_f1 = val_f1
        best_pred = val_pred
        torch.save(model.state_dict(), "best_complex.pt")


tensor([[9.6863e-03],
        [3.4976e-04],
        [1.0000e+00],
        ...,
        [3.7682e-04],
        [3.5048e-04],
        [3.5691e-04]], grad_fn=<RsubBackward1>)
Iteration 0: 0.9573822617530823
val loss tensor(0.8644)
f1 w/ 0.5 threshold 0.5389048163593573
Iteration 1: 0.7983121871948242
val loss tensor(0.7087)
f1 w/ 0.5 threshold 0.5500277705571773
Iteration 2: 0.6616075038909912
val loss tensor(0.5831)
f1 w/ 0.5 threshold 0.5611543117468767
Iteration 3: 0.5385414958000183
val loss tensor(0.4938)
f1 w/ 0.5 threshold 0.5713164135102224
Iteration 4: 0.45025917887687683
val loss tensor(0.4145)
f1 w/ 0.5 threshold 0.5832352587535825
Iteration 5: 0.3785131275653839
val loss tensor(0.3597)
f1 w/ 0.5 threshold 0.5946052934809039
Iteration 6: 0.3310108780860901
val loss tensor(0.3109)
f1 w/ 0.5 threshold 0.6049645533978529
Iteration 7: 0.2867151200771332
val loss tensor(0.2659)
f1 w/ 0.5 threshold 0.6171883180043028
Iteration 8: 0.25164327025413513
val loss tensor(0.2293)
f1 w/ 0.5 t

val loss tensor(0.0646)
f1 w/ 0.5 threshold 0.7502145857328353
Iteration 85: 0.06548275798559189
val loss tensor(0.0645)
f1 w/ 0.5 threshold 0.7502145857328353
Iteration 86: 0.0653853565454483
val loss tensor(0.0644)
f1 w/ 0.5 threshold 0.7502145857328353
Iteration 87: 0.06529440730810165
val loss tensor(0.0643)
f1 w/ 0.5 threshold 0.7502145857328353
Iteration 88: 0.06520730257034302
val loss tensor(0.0642)
f1 w/ 0.5 threshold 0.7502145857328353
Iteration 89: 0.06512439996004105
val loss tensor(0.0641)
f1 w/ 0.5 threshold 0.7528387439472204
Iteration 90: 0.06504955887794495
val loss tensor(0.0641)
f1 w/ 0.5 threshold 0.7528387439472204
Iteration 91: 0.0649760365486145
val loss tensor(0.0640)
f1 w/ 0.5 threshold 0.7528387439472204
Iteration 92: 0.06483722478151321
val loss tensor(0.0639)
f1 w/ 0.5 threshold 0.7528387439472204
Iteration 93: 0.06472838670015335
val loss tensor(0.0639)
f1 w/ 0.5 threshold 0.7528387439472204
Iteration 94: 0.06463892012834549
val loss tensor(0.0638)
f1 w/ 0.

val loss tensor(0.0580)
f1 w/ 0.5 threshold 0.7610366540467318
Iteration 169: 0.059609394520521164
val loss tensor(0.0579)
f1 w/ 0.5 threshold 0.7610366540467318
Iteration 170: 0.05957398936152458
val loss tensor(0.0579)
f1 w/ 0.5 threshold 0.7610366540467318
Iteration 171: 0.0595400333404541
val loss tensor(0.0578)
f1 w/ 0.5 threshold 0.7610366540467318
Iteration 172: 0.059509534388780594
val loss tensor(0.0578)
f1 w/ 0.5 threshold 0.7610366540467318
Iteration 173: 0.059479229152202606
val loss tensor(0.0577)
f1 w/ 0.5 threshold 0.7610366540467318
Iteration 174: 0.05945027247071266
val loss tensor(0.0577)
f1 w/ 0.5 threshold 0.7610366540467318
Iteration 175: 0.0594213642179966
val loss tensor(0.0576)
f1 w/ 0.5 threshold 0.7610366540467318
Iteration 176: 0.05939330905675888
val loss tensor(0.0576)
f1 w/ 0.5 threshold 0.7610366540467318
Iteration 177: 0.05936622619628906
val loss tensor(0.0576)
f1 w/ 0.5 threshold 0.7610366540467318
Iteration 178: 0.05933916196227074
val loss tensor(0.0

In [155]:
# tune on val set

print("Val -- The best f1 is {} w/ naive threshold 0.5".format(best_val_f1))

best_tuned_threshold = 0.5
best_tuned_f1 = best_val_f1

for threshold_ in np.linspace(0.0, 1.0, num=1000):
    y_val_preds = best_pred >= threshold_
    prec, recall, f1, _ = precision_recall_fscore_support(y_val, y_val_preds, average='macro')
    if f1 > best_tuned_f1:
        best_tuned_threshold = threshold_
        best_tuned_f1 = f1
print("Val -- After tuning, the best f1 is {} w/ threshold {}".format(best_tuned_f1, best_tuned_threshold))


bestModel = ComplexRuleLNN(0.8, 2, False)
bestModel.load_state_dict(torch.load("best_complex.pt"))
bestModel.eval()

with torch.no_grad():
    test_pred = bestModel(x_test, m_labels_test)
    test_pred = test_pred >= best_tuned_threshold
    prec, recall, f1, _ = precision_recall_fscore_support(y_test, test_pred, average='macro')
    print("Test -- f1 is {} w/ threshold {}".format(f1, best_tuned_threshold))

Val -- The best f1 is 0.7638835403228394 w/ naive threshold 0.5
Val -- After tuning, the best f1 is 0.7856135050703569 w/ threshold 0.992992992992993
Test -- f1 is 0.8164090967900519 w/ threshold 0.992992992992993


In [164]:
for name1, mod1 in bestModel.named_children():
    for name, mod in mod1.named_children():
        print("========={}=========".format(name))
        if name == 'sim_disjunction_or_ops':
            for each_op in mod:
                print(each_op.AND.cdd())
        elif name == 'predicate_and_ops':
            for each_op in mod:
                print(each_op.cdd())
        elif 'and' in name:
            print(name, mod.cdd())
        else:
            print(name, mod.AND.cdd())
#     if 'and' in name1:
#         print(name1, mod1.cdd())
#     elif 'or' in name1:
#         print(name1, mod1.AND.cdd())

=========sim_disjunction_or_ops=========
(tensor(5.1019, grad_fn=<SelectBackward>), tensor([10.4697,  8.0609], grad_fn=<SliceBackward>))
(tensor(5.7117, grad_fn=<SelectBackward>), tensor([ 8.2640, 11.7477], grad_fn=<SliceBackward>))
(tensor(6.4294, grad_fn=<SelectBackward>), tensor([12.9908,  8.3084], grad_fn=<SliceBackward>))
(tensor(3.8584, grad_fn=<SelectBackward>), tensor([6.8998, 6.3426], grad_fn=<SliceBackward>))
=========predicate_and=========
predicate_and (tensor(4.6245, grad_fn=<SelectBackward>), tensor([13.1790,  5.6998], grad_fn=<SliceBackward>))
=========sim_disjunction_or_ops=========
(tensor(6.3645, grad_fn=<SelectBackward>), tensor([13.3272,  9.0812], grad_fn=<SliceBackward>))
(tensor(6.9162, grad_fn=<SelectBackward>), tensor([14.1029,  9.5137], grad_fn=<SliceBackward>))
(tensor(6.0238, grad_fn=<SelectBackward>), tensor([13.0749,  7.8272], grad_fn=<SliceBackward>))
(tensor(4.1672, grad_fn=<SelectBackward>), tensor([6.7004, 7.7094], grad_fn=<SliceBackward>))
=========pre

In [165]:
for name, param in bestModel.named_parameters():
    print(name, param)

pureNameRule.sim_disjunction_or_ops.0.AND.cdd.gamma Parameter containing:
tensor([[0.6911]], requires_grad=True)
pureNameRule.sim_disjunction_or_ops.0.AND.cdd.mu Parameter containing:
tensor([[1.5432, 0.1545, 0.8296]], requires_grad=True)
pureNameRule.sim_disjunction_or_ops.1.AND.cdd.gamma Parameter containing:
tensor([[0.4595]], requires_grad=True)
pureNameRule.sim_disjunction_or_ops.1.AND.cdd.mu Parameter containing:
tensor([[-0.3108,  1.7886,  1.6416]], requires_grad=True)
pureNameRule.sim_disjunction_or_ops.2.AND.cdd.gamma Parameter containing:
tensor([[0.6444]], requires_grad=True)
pureNameRule.sim_disjunction_or_ops.2.AND.cdd.mu Parameter containing:
tensor([[ 1.9484, -1.4607,  2.6722]], requires_grad=True)
pureNameRule.sim_disjunction_or_ops.3.AND.cdd.gamma Parameter containing:
tensor([[0.1029]], requires_grad=True)
pureNameRule.sim_disjunction_or_ops.3.AND.cdd.mu Parameter containing:
tensor([[0.4294, 0.0292, 0.2391]], requires_grad=True)
pureNameRule.predicate_and.cdd.gamma P

In [109]:

for each_op in bestModel.sim_disjunction_or_ops:
    #lets check the LNN conjunction parameters post-training
    #do these look different from the pre-training settings?
    beta, argument_wts = each_op.AND.cdd()
    print("beta (post-training): " + str(beta.item()))
    print("argument weights (post-training): " + str(argument_wts.detach()))

for each_op in bestModel.sim_disjunction_or_ops:
    #lets check the LNN conjunction parameters post-training
    #do these look different from the pre-training settings?
    beta, argument_wts = each_op.AND.cdd()
    print("beta (post-training): " + str(beta.item()))
    print("argument weights (post-training): " + str(argument_wts.detach()))

beta (post-training): 4.196691513061523
argument weights (post-training): tensor([7.9233, 7.2595])
beta (post-training): 3.902745246887207
argument weights (post-training): tensor([6.9828, 6.3890])
beta (post-training): 4.169642925262451
argument weights (post-training): tensor([7.6434, 7.0587])
beta (post-training): 4.3188300132751465
argument weights (post-training): tensor([7.2704, 8.2539])
beta (post-training): 4.196691513061523
argument weights (post-training): tensor([7.9233, 7.2595])
beta (post-training): 3.902745246887207
argument weights (post-training): tensor([6.9828, 6.3890])
beta (post-training): 4.169642925262451
argument weights (post-training): tensor([7.6434, 7.0587])
beta (post-training): 4.3188300132751465
argument weights (post-training): tensor([7.2704, 8.2539])


## Code for XOR example

In [136]:
#to train a xor we need its truth table
x = torch.from_numpy(np.array([[0, 0], \
                               [0, 1], \
                               [1, 0], \
                               [1, 1]])).float()

#the target values for each row in the truth table (xor)
y = torch.from_numpy(np.array([[0], \
                               [1], \
                               [1], \
                               [0]])).float()

In [140]:
class xorLNN(nn.Module):
    def __init__(self, alpha, arity, slack):
        super(xorLNN, self).__init__()
        self.op_and1 = and_lukasiewicz(alpha, arity, slack)
        self.op_and2 = and_lukasiewicz(alpha, arity, slack)
        self.op_or = or_lukasiewicz(alpha, arity, slack)
    
    def forward(self, x):
        x0 = x[:,0].view(-1,1)
        print(x0)
        x1 = x[:,1].view(-1,1)
        print(x1)
        print(torch.cat((x0, negation(x1)), 1))
        yhat = self.op_or(torch.cat((self.op_and1(torch.cat((x0, negation(x1)), 1)), \
                            self.op_and2(torch.cat((negation(x0), x1), 1))), 1))
        return yhat

In [141]:
model = xorLNN(0.8, 2, False)
model(x)

tensor([[0.],
        [0.],
        [1.],
        [1.]])
tensor([[0.],
        [1.],
        [0.],
        [1.]])
tensor([[0., 1.],
        [0., 0.],
        [1., 1.],
        [1., 0.]])


tensor([[4.6349e-04],
        [9.9932e-01],
        [9.9967e-01],
        [4.6349e-04]], grad_fn=<RsubBackward1>)

In [25]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [26]:
for iter in range(100):
    model.train()
    optimizer.zero_grad()

    yhat = model(x)
    loss = loss_fn(yhat, y)

    print("Iteration " + str(iter) + ": " + str(loss.item()))
    
    loss.backward()
    optimizer.step()


Iteration 0: 0.00041395798325538635
Iteration 1: 0.0003413597878534347
Iteration 2: 0.00027990160742774606
Iteration 3: 0.00022847841319162399
Iteration 4: 0.00018598556926008314
Iteration 5: 0.00015122962940949947
Iteration 6: 0.0001230025663971901
Iteration 7: 0.00010024568473454565
Iteration 8: 8.197502756956965e-05
Iteration 9: 6.734087219228968e-05
Iteration 10: 5.5627755500609055e-05
Iteration 11: 4.625439760275185e-05
Iteration 12: 3.872895103995688e-05
Iteration 13: 3.267884312663227e-05
Iteration 14: 2.7761290766648017e-05
Iteration 15: 2.3767666789353825e-05
Iteration 16: 2.051913361356128e-05
Iteration 17: 1.7821967048803344e-05
Iteration 18: 1.5646355677745305e-05
Iteration 19: 1.3783679605694488e-05
Iteration 20: 1.2263739336049184e-05
Iteration 21: 1.0967321941279806e-05
Iteration 22: 9.89442560239695e-06
Iteration 23: 8.970543603936676e-06
Iteration 24: 8.165872714016587e-06
Iteration 25: 7.480413842131384e-06
Iteration 26: 6.899264008097816e-06
Iteration 27: 6.407521595

In [28]:
#this is a hyperparameter
alpha = 0.8

op_and1 = and_lukasiewicz(alpha, 2, False)
op_and2 = and_lukasiewicz(alpha, 2, False)
op_or = or_lukasiewicz(alpha, 2, False)

#to train a xor we need its truth table
x = torch.from_numpy(np.array([[0, 0], \
                               [0, 1], \
                               [1, 0], \
                               [1, 1]])).float()

#the target values for each row in the truth table (xor)
y = torch.from_numpy(np.array([[0], \
                               [1], \
                               [1], \
                               [0]])).float()

loss_fn = nn.BCELoss()
optimizer = optim.Adam([{'params': op_or.parameters()}, \
                        {'params': op_and1.parameters()}, \
                        {'params': op_and2.parameters()}], lr=0.1)

for iter in range(100):
    op_or.train()
    op_and1.train()
    op_and2.train()
    optimizer.zero_grad()

    x0 = x[:,0].view(-1,1)
    x1 = x[:,1].view(-1,1)
    yhat = op_or(torch.cat((op_and1(torch.cat((x0, negation(x1)), 1)), \
                            op_and2(torch.cat((negation(x0), x1), 1))), 1))
    loss = loss_fn(yhat, y)

    print("Iteration " + str(iter) + ": " + str(loss.item()))
    loss.backward()
    optimizer.step()

#check to see output of xor post-training
x0 = x[:,0].view(-1,1)
x1 = x[:,1].view(-1,1)
yhat = op_or(torch.cat((op_and1(torch.cat((x0, negation(x1)), 1)), \
                        op_and2(torch.cat((negation(x0), x1), 1))), 1))
check_values = torch.cat((yhat, y), 1)
print("------- Checking outputs (left) vs ground truth (right): -----")
print(check_values.detach())

#LNN parameters: post-training (we have 3 sets of beta, argument weights)
print("--------------- LNN Parameters (post-training) ---------------")
beta_or, argument_wts_or = op_or.AND.cdd()
beta_and1, argument_wts_and1 = op_and1.cdd()
beta_and2, argument_wts_and2 = op_and2.cdd()

np.set_printoptions(precision=3, suppress=True)
print("OR (beta, argument weights): " \
      + str(np.around(beta_or.item(), decimals=3)) + " " \
      + str(argument_wts_or.detach().numpy()))
print("AND1 (beta, argument weights): " \
      + str(np.around(beta_and1.item(), decimals=3)) + " " \
      + str(argument_wts_and1.detach().numpy()))
print("AND2 (beta, argument weights): " \
      + str(np.around(beta_and2.item(), decimals=3)) + " " \
      + str(argument_wts_and2.detach().numpy()))

Iteration 0: 0.0005070384358987212
Iteration 1: 0.0004226093296892941
Iteration 2: 0.00035011349245905876
Iteration 3: 0.0002885941066779196
Iteration 4: 0.0002370504371356219
Iteration 5: 0.00019430331303738058
Iteration 6: 0.00015917410200927407
Iteration 7: 0.00013051435234956443
Iteration 8: 0.0001072355080395937
Iteration 9: 8.839827933115885e-05
Iteration 10: 7.324236503336579e-05
Iteration 11: 6.0992642829660326e-05
Iteration 12: 5.106781463837251e-05
Iteration 13: 4.3065447243861854e-05
Iteration 14: 3.6598037695512176e-05
Iteration 15: 3.130791810690425e-05
Iteration 16: 2.6941725081996992e-05
Iteration 17: 2.3410046196659096e-05
Iteration 18: 2.048934402409941e-05
Iteration 19: 1.8030596038443036e-05
Iteration 20: 1.6003998098312877e-05
Iteration 21: 1.4275432477006689e-05
Iteration 22: 1.2785292710759677e-05
Iteration 23: 1.1593181625357829e-05
Iteration 24: 1.0550087608862668e-05
Iteration 25: 9.641104952606838e-06
Iteration 26: 8.851335223880596e-06
Iteration 27: 8.1807766

In [ ]:
# arity should be 2

In [150]:
# class PureNameLNN(nn.Module):
#     def __init__(self, alpha, sim_arity=4, rule_arity=2, slack=None):
#         super(PureNameLNN, self).__init__()
#         self.threshold = 0.5
        
#         self.sim_disjunction_or = or_lukasiewicz(alpha, sim_arity, slack)
    
#     def forward(self, x, mention_labels=None):
#         """
#             x: scores['jw'], scores['jacc'], scores['lev'], scores['spacy'], 
#                normalized_ref_scores[ref_idx], normalized_ctx_scores[ctx_idx]
#         """
#         yhat = None
        
#         # RULE 1: lookup predicate
#         lookup_features = x[:,5]
#         print("lookup_features", lookup_features)
        
#         # RULE 2: similarity predicate(mention==label AND Jacc(m, lb) AND Lev(m, lb) AND Jaro(m, lb))
#         feature_list = []
#         # rule 2 (1) mention==label
#         mentions = np.array([m[0].lower() for m in mention_labels])
#         labels = np.array([m[1].lower() for m in mention_labels])
#         exact_match_features = torch.from_numpy(np.array(mentions == labels).astype(float)).float()
#         feature_list.append(exact_match_features)
#         print("exact_match_features", exact_match_features)
        
#         # rule 2 (2)-(4) Jaro(m, lb) AND Jacc(m, lb) AND Lev(m, lb))
#         sim_features = x[:, 0:3]
#         print(sim_features)

#         return yhat